In [ ]:
%load_ext autoreload

In [ ]:
%autoreload
from reappraisalmodel.lightningreapp import LightningReapp
from reappraisalmodel.ldhdata import LDHDataModule
import datetime
import pytorch_lightning as lit
import torch
from torch.utils.data import DataLoader

# Define constants


Bucket = 'ldhdata'
# Match on reapp_model/far_* or obj_*

models_to_evaluate = []
save_dir = './lightning_logs/results-obj_20210306_194820'

STRAT = 'obj'
BATCH_SIZE = 128

#model = LightningReapp.load_from_checkpoint("s3://ldhdata/reapp_model/far_20210306_231630/checkpoints/epoch=41-step=6887.ckpt", map_location=torch.device('cpu'))
model = LightningReapp.load_from_checkpoint("s3://ldhdata/reapp_model/obj_20210306_194820/checkpoints/epoch=43-step=7215.ckpt", map_location=torch.device('cpu'))

ldhdata = LDHDataModule(data_dir='.', strat=STRAT)
ldhdata.load_train_data()
ldhdata.load_eval_data()



UsageError: Line magic function `%autoreload` not found.


In [ ]:
%autoreload

from pytorch_lightning.loggers import CSVLogger
from pl_bolts.callbacks.printing import PrintTableMetricsCallback
callback = PrintTableMetricsCallback()

ldhdata.train_data.set_format(type='torch', columns=['input_ids', 'attention_mask'])
train_data = ldhdata.train_data

ldhdata.eval_data.set_format(type='torch', columns=['input_ids', 'attention_mask'])
eval_data = ldhdata.eval_data 

train_dl = DataLoader(train_data, batch_size=BATCH_SIZE, num_workers=4)
eval_dl = DataLoader(eval_data, batch_size=BATCH_SIZE, num_workers=4)

csvlogger = CSVLogger(
    save_dir = save_dir
)

today = datetime.datetime.today().strftime('%Y%m%d_%H%M%S')
trainer = lit.Trainer(
    logger=csvlogger,
    callbacks=[callback],
    profiler='simple',
    limit_test_batches=0.30,
)


GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [ ]:
from datasets import Dataset
import pandas as pd
import numpy as np


trainresults = trainer.test(
    model,
    test_dataloaders=train_dl,
    verbose=True
)

ldhdata.train_data.reset_format()
predictions = trainresults[0]['prediction']
df = pd.DataFrame(ldhdata.train_data.select(range(len(predictions))), columns=['response', 'score'], dtype=np.float32)
df['predicted'] = predictions
df.to_csv(save_dir + "/study1preds.csv")

Testing:   0%|          | 0/30 [04:04<?, ?it/s]


In [ ]:
from sklearn import metrics
import numpy as np
import pandas as pd

study2results = trainer.test(model, eval_dl, verbose=True)

ldhdata.eval_data.reset_format()


Testing: 100%|██████████| 75/75 [59:03<00:00, 47.24s/it]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'prediction': array([4.8511314, 4.4780025, 3.540176 , ..., 5.9349713, 5.883519 ,
       5.5979033], dtype=float32)}
--------------------------------------------------------------------------------


prediction
──────────
[4.8511314 4.4780025 3.540176  ... 5.9349713 5.883519  5.5979033]


NameError: name 'pd' is not defined

In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics

predictions = study2results[0]['prediction']

iidf = pd.DataFrame(ldhdata.eval_data.select(range(len(predictions))), columns=['response', 'score'], dtype=np.float32)

iidf['predicted'] = predictions
iidf.to_csv(save_dir + "/study2preds.csv")
# print("EV:", metrics.explained_variance_score(iidf['score'], iidf['predicted']))
# print("r:" , np.sqrt(metrics.r2_score(iidf['score'], iidf['predicted'])))
# print("Summary:\n", iidf.describe())


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
iidf

,response,score,predicted
0,this is a photo of a man and woman who look li...,NaN,4.851131
1,the man and woman a. this photo is of a child ...,NaN,4.478003
2,the child has a pained expression on its face ...,NaN,3.540176
3,this is two hands holding a small gun.,NaN,6.077178
4,"the hands have long nails, and the gun is eith...",NaN,5.982574
...,...,...,...
9595,Picture of a woman screaming at a man who‚Äö√Ñ...,NaN,3.615735
9596,Two young adults or teenagers are sitting on t...,NaN,3.211668
9597,Picture of a man holding a young child on his ...,NaN,5.934971
9598,Picture of a man looking down with his arms cr...,NaN,5.883519
